In [1]:
import cv2
import glob
from skimage.feature import hog

import time
start = time.time()
# read training files and use hog function to get the feature vector
train_hog=[]
for file in sorted(glob.glob('traffic/traffic-small/train/*.jpg')):
    img = cv2.imread(file,0)
    #img = cv2.resize(img, (70, 35))
    img = cv2.resize(img, (80, 40))
    h = hog(img, block_norm='L2')
    train_hog.append(h)
    
end=time.time()
print('time:',end-start)

time: 7.735548973083496


In [2]:
# read labels of training dataset
import numpy as np
train_label=[]
with open('traffic/traffic-small/train.labels', mode='r') as f:
    for line in f:
        train_label.append(int(line))
train_label=np.array(train_label)

In [3]:
# standardise the data
start = time.time()

from sklearn.preprocessing import StandardScaler
train_hog_std = StandardScaler().fit_transform(train_hog)
train_hog_std.shape

end=time.time()
print('time:',end-start)


time: 0.5468509197235107


In [36]:
'''
# implementing PCA method to reduce dimension
start = time.time()

from sklearn import decomposition
pca = decomposition.PCA(n_components=100)
train_pca = pca.fit_transform(train_hog_std)
train_pca.shape

end=time.time()
print('time:',end-start)'''

time: 0.880789041519165


In [4]:
train_hog_std.shape

(4209, 1944)

In [5]:
# training SVM classifier
start = time.time()

from sklearn import svm

svr = svm.SVC()
#parameters = {'kernel': ['rbf'], 'C':[1,10,100], 'gamma':[ 1e-4, 1e-3, 1e-2]}
parameters = {'kernel': ['rbf'], 'C':[3], 'gamma':[ (5e-4)-(1e-5), 5e-4, (5e-4)+(1e-5)]}
clf = GridSearchCV(svr, parameters, scoring='f1_micro',cv=5, n_jobs=-1)    #choose the best parameter to used in SVM classifier with C.V.=8
#clf = svm.SVC(C=3, kernel='rbf',gamma=0.0005)
clf.fit(train_hog_std, train_label)

end=time.time()
print('time:',end-start)

time: 63.31363105773926


In [119]:
print("img = cv2.resize(img, (80, 40))")
print("{'kernel': ['rbf'], 'C':[3], 'gamma':[(5e-4)-(1e-5), 5e-4, (5e-4)+(1e-5)]}")
print(clf.best_params_)

img = cv2.resize(img, (80, 40))
{'kernel': ['rbf'], 'C':[3], 'gamma':[ 3e-4, 5e-4, 7e-4]}
{'C': 3, 'gamma': 0.0005, 'kernel': 'rbf'}


In [116]:
print("img = cv2.resize(img, (80, 40))")
print("{'kernel': ['rbf'], 'C':[3,5,10], 'gamma':[3e-4, 5e-4, 7e-4]}")
print(clf.best_params_)

img = cv2.resize(img, (80, 40))
{'kernel': ['rbf'], 'C':[3,5,10], 'gamma':[ 5e-4, 1e-3, 5e-3]}
{'C': 3, 'gamma': 0.0005, 'kernel': 'rbf'}


In [7]:
start = time.time()

# applying SVM to get predicted label
train_y_pred = clf.predict(train_hog_std)

end=time.time()
print('time:',end-start)

time: 50.474985122680664


In [8]:
from sklearn.metrics import f1_score
f1_score(train_label, train_y_pred, average='micro') 

0.9961986220004752

In [9]:
start = time.time()

# read test file
test_hog=[]
for file in sorted(glob.glob('traffic/traffic/test/*.jpg')):
    
    img = cv2.imread(file,0)
    img = cv2.resize(img, (80,40))
    h = hog(img, block_norm='L2')
    test_hog.append(h)

# standardise the data
test_hog_std = StandardScaler().fit_transform(test_hog)
test_hog_std.shape
'''
# implementing PCA method to reduce dimension
from sklearn import decomposition
pca = decomposition.PCA(n_components=100)
test_pca = pca.fit_transform(test_hog_std)
test_pca.shape'''

end=time.time()
print('time:',end-start)

time: 218.88489484786987


In [10]:
start = time.time()

# applying SVM to get predicted label
test_y_pred = clf.predict(test_hog_std)

end=time.time()
print('time:',end-start)

time: 859.1795790195465


In [11]:
len(test_y_pred)

100000

In [12]:
# output label file
import csv
with open('prediction.dat', mode='w') as out:
    writer = csv.writer(out)
    for i in range(len(test_y_pred)):
        writer.writerow([test_y_pred[i]])